In [ ]:
'''Online Retail Analytics – Final Comprehensive Analysis

Business Theme: Inventory Optimization & Customer Retention Strategy  

This notebook analyzes a real-world online retail dataset to:
- Clean and explore transaction data
- Visualize geographic and temporal sales patterns
- Build RFM-based customer segments
- Statistically test key business hypotheses (H1–H5)
- Cluster customers into behavioral groups
- Predict which customers are likely to return in the next 3 months '''


In [ ]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import datetime as dt
from scipy import stats
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn.mixture import GaussianMixture
from sklearn.metrics import silhouette_score, davies_bouldin_score
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, roc_auc_score, accuracy_score, roc_curve
import plotly.express as px
import os


DATA_PATH = "Online_Retail.xlsx"

# Configuration
warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

# Create output directories
os.makedirs('figures', exist_ok=True)
# os.makedirs('results', exist_ok=True)

print("="*80)
print("ONLINE RETAIL ANALYTICS - FINAL ANALYSIS")
print("Business Theme: Inventory Optimization & Customer Retention Strategy")
print("="*80)


In [ ]:
'''1. Data Loading and Cleaning

We first load the Online Retail dataset, inspect basic data quality, remove duplicates, 
and create a clean **sales dataset** with only valid, positive-quantity transactions.'''

In [ ]:
print("\n" + "="*80)
print("SECTION 1: DATA LOADING AND CLEANING")
print("="*80)

# Load data
try:
    df = pd.read_excel(DATA_PATH)
except FileNotFoundError:
    print("Error: 'Online_Retail.xlsx' not found. Please ensure file is in directory.")
    raise

# --- Data Quality Report ---
print("\n--- Data Quality Report ---")
print(f"Total rows: {len(df):,}")
print(f"Missing CustomerID: {df['CustomerID'].isna().sum():,} ({df['CustomerID'].isna().sum()/len(df)*100:.2f}%)")
print(f"Missing Description: {df['Description'].isna().sum():,}")
print(f"Negative Quantity (returns): {(df['Quantity'] < 0).sum():,}")
print(f"Unique products: {df['StockCode'].nunique():,}")
print(f"Countries: {df['Country'].nunique()}")

# Data Cleaning
df.drop_duplicates(inplace=True)
df_clean = df.dropna(subset=['CustomerID'])

# Create Sales Dataset (Positive quantities only) for General Analysis
df_sales = df_clean[(df_clean['Quantity'] > 0) & (df_clean['UnitPrice'] > 0)].copy()

# Feature Engineering
df_sales['TotalAmount'] = df_sales['Quantity'] * df_sales['UnitPrice']
df_sales['InvoiceDate'] = pd.to_datetime(df_sales['InvoiceDate'])
df_sales['Date'] = df_sales['InvoiceDate'].dt.date
df_sales['Month'] = df_sales['InvoiceDate'].dt.month
df_sales['Year'] = df_sales['InvoiceDate'].dt.year
df_sales['DayOfWeek'] = df_sales['InvoiceDate'].dt.day_name()

print(f"\nFinal clean dataset for sales analysis: {df_sales.shape}")
df_sales.head()

In [ ]:
'''2. Enhanced Data Visualization

We explore:
- Geographic revenue distribution across countries
- Temporal revenue patterns using daily sales and 30-day moving average'''

In [ ]:
print("\n" + "="*80)
print("SECTION 2: ENHANCED DATA VISUALIZATION")
print("="*80)

# 2.1 Geographic Visualization
country_sales = df_sales.groupby('Country').agg({
    'TotalAmount': 'sum'
}).reset_index().sort_values('TotalAmount', ascending=False)

fig = px.choropleth(
    country_sales,
    locations='Country',
    locationmode='country names',
    color='TotalAmount',
    title='Global Revenue Distribution',
    color_continuous_scale='Viridis'
)
fig.show()
fig.write_html('figures/geographic_revenue_map.html')
print("✓ Geographic revenue map created")

# 2.2 Top 10 Countries by Revenue
top_countries = country_sales.head(10)

plt.figure(figsize=(12, 6))
sns.barplot(data=top_countries, x='TotalAmount', y='Country', palette='viridis')
plt.title('Top 10 Countries by Total Revenue', fontsize=14, fontweight='bold')
plt.xlabel('Revenue (£)')
plt.ylabel('Country')
plt.tight_layout()
plt.savefig('figures/top10_countries_revenue.png', dpi=300)
plt.show()

print("Top 10 revenue countries visualized")

# 2.3 Time Series Analysis
daily_sales = df_sales.groupby('Date')['TotalAmount'].sum().reset_index()
daily_sales.set_index('Date', inplace=True)

plt.figure(figsize=(15, 6))
plt.plot(daily_sales.index, daily_sales['TotalAmount'], color='#2E86AB', label='Daily Revenue')
plt.plot(
    daily_sales.index,
    daily_sales['TotalAmount'].rolling(window=30).mean(),
    color='#C73E1D',
    linewidth=2,
    label='30-day MA'
)
plt.title('Daily Revenue & Moving Average', fontsize=14, fontweight='bold')
plt.legend()
plt.tight_layout()
plt.savefig('figures/time_series_analysis.png', dpi=300)
plt.show()
print("Time series analysis created")

# 2.4 Monthly Revenue Trend
monthly_sales = df_sales.groupby(['Year', 'Month'])['TotalAmount'].sum().reset_index()
monthly_sales['YearMonth'] = monthly_sales['Year'].astype(str) + '-' + monthly_sales['Month'].astype(str)

plt.figure(figsize=(14, 6))
sns.lineplot(data=monthly_sales, x='YearMonth', y='TotalAmount', marker='o')
plt.xticks(rotation=45)
plt.title('Monthly Revenue Trend (Seasonality Highlight)', fontsize=14, fontweight='bold')
plt.xlabel('Month')
plt.ylabel('Revenue (£)')
plt.tight_layout()
plt.savefig('figures/monthly_revenue_trend.png', dpi=300)
plt.show()

print("Monthly seasonal sales trend created")



In [ ]:
'''3. RFM Analysis and Customer Segmentation

We compute:
- **Recency** (days since last purchase)  
- **Frequency** (number of invoices)  
- **Monetary** (total spend)

Then we build an RFM score and map customers to segments like **Champions**, **Loyal**, **At Risk**, etc.'''

In [ ]:
print("\n" + "="*80)
print("SECTION 3: RFM ANALYSIS")
print("="*80)

latest_date = df_sales['InvoiceDate'].max() + dt.timedelta(days=1)
rfm = df_sales.groupby('CustomerID').agg({
    'InvoiceDate': lambda x: (latest_date - x.max()).days,
    'InvoiceNo': 'nunique',
    'TotalAmount': 'sum'
}).reset_index()
rfm.columns = ['CustomerID', 'Recency', 'Frequency', 'Monetary']

# Segmentation Logic
rfm['R_Score'] = pd.qcut(rfm['Recency'], q=4, labels=[4, 3, 2, 1])
rfm['F_Score'] = pd.qcut(rfm['Frequency'].rank(method='first'), q=4, labels=[1, 2, 3, 4])
rfm['M_Score'] = pd.qcut(rfm['Monetary'], q=4, labels=[1, 2, 3, 4])
rfm['RFM_Sum'] = rfm['R_Score'].astype(int) + rfm['F_Score'].astype(int) + rfm['M_Score'].astype(int)

def segment_customer(score):
    if score >= 9:
        return 'Champions'
    elif score >= 8:
        return 'Loyal Customers'
    elif score >= 6:
        return 'Potential Loyalists'
    elif score >= 5:
        return 'At Risk'
    else:
        return 'Hibernating/Lost'

rfm['Segment'] = rfm['RFM_Sum'].apply(segment_customer)
print(rfm['Segment'].value_counts())

rfm.head()

# 3.4 Segment Distribution Visualization
plt.figure(figsize=(8, 8))
rfm['Segment'].value_counts().plot(kind='pie', autopct='%1.1f%%', startangle=90, colors=sns.color_palette('husl'))
plt.title('Customer Segment Distribution')
plt.ylabel('')
plt.tight_layout()
plt.savefig('figures/rfm_segment_pie.png', dpi=300)
plt.show()

print("✓ Customer segment distribution chart created")



In [ ]:
'''4. Statistical Hypothesis Testing (H1 – H5)

We test five business hypotheses:

- **H1:** High-frequency customers have higher monetary value  
- **H2:** Repeat customers have higher average order value than single-purchase customers  
- **H3:** Transaction amounts differ by day of week  
- **H4:** International customers have higher order value than UK customers  
- **H5:** Return rates differ significantly across countries  
'''

In [ ]:
print("\n" + "="*80)
print("SECTION 4: STATISTICAL HYPOTHESIS TESTING")
print("="*80)

# --- H1: Frequency-Monetary Relationship ---
print("\n--- H1: High-frequency vs Low-frequency Monetary Value ---")
median_freq = rfm['Frequency'].median()
high_freq = rfm[rfm['Frequency'] >= median_freq]['Monetary']
low_freq = rfm[rfm['Frequency'] < median_freq]['Monetary']
t_stat, p_val = stats.ttest_ind(high_freq, low_freq)
print(f"High Freq Mean: £{high_freq.mean():.2f} | Low Freq Mean: £{low_freq.mean():.2f}")
print(f"T-statistic: {t_stat:.4f}, P-value: {p_val:.4e}")
print(f"Result: {'REJECT' if p_val < 0.05 else 'FAIL TO REJECT'} null hypothesis")

# --- H2: Repeat vs Single Purchase Average Order Value (MATCHING REPORT) ---
print("\n--- H2: Repeat vs Single Purchase Average Order Value ---")

# Average value of line items per customer
customer_stats = df_sales.groupby('CustomerID').agg({
    'InvoiceNo': 'nunique',
    'TotalAmount': 'mean'  # average line-item value (Quantity * UnitPrice)
}).reset_index()

customer_stats.columns = ['CustomerID', 'NumOrders', 'AvgLineValue']

repeat_customers = customer_stats[customer_stats['NumOrders'] > 1]['AvgLineValue']
single_customers = customer_stats[customer_stats['NumOrders'] == 1]['AvgLineValue']

t_stat, p_val = stats.ttest_ind(repeat_customers, single_customers)
print(f"Repeat customers avg value: £{repeat_customers.mean():.2f} (n={len(repeat_customers):,} customers)")
print(f"Single customers avg value: £{single_customers.mean():.2f} (n={len(single_customers):,} customers)")
print(f"T-statistic: {t_stat:.4f}, P-value: {p_val:.4f}")
print(f"Result: {'REJECT' if p_val < 0.05 else 'FAIL TO REJECT'} null hypothesis")

# --- H3: Weekly Transaction Patterns ---
print("\n--- H3: Weekly Transaction Patterns (ANOVA) ---")
present_days = df_sales['DayOfWeek'].unique()
day_groups = []
valid_days = []

for d in present_days:
    group_data = df_sales[df_sales['DayOfWeek'] == d]['TotalAmount']
    if len(group_data) > 10:
        day_groups.append(group_data)
        valid_days.append(d)

print(f"Days included in ANOVA: {valid_days}")

if len(day_groups) >= 2:
    f_stat, p_val = stats.f_oneway(*day_groups)
    print(f"F-statistic: {f_stat:.4f}, P-value: {p_val:.4e}")
    print(f"Result: {'REJECT' if p_val < 0.05 else 'FAIL TO REJECT'} null hypothesis")
else:
    print("Insufficient groups for ANOVA.")

# --- H4: Domestic vs International ---
print("\n--- H4: UK vs International Order Values ---")
uk_orders = df_sales[df_sales['Country'] == 'United Kingdom']['TotalAmount']
intl_orders = df_sales[df_sales['Country'] != 'United Kingdom']['TotalAmount']
t_stat, p_val = stats.ttest_ind(intl_orders, uk_orders, equal_var=False)
print(f"Intl Mean: £{intl_orders.mean():.2f} | UK Mean: £{uk_orders.mean():.2f}")
print(f"T-statistic: {t_stat:.4f}, P-value: {p_val:.4e}")
print(f"Result: {'REJECT' if p_val < 0.05 else 'FAIL TO REJECT'} null hypothesis")

# --- H5: Country-Level Return Rates ---
print("\n--- H5: Country-Level Return Rates (Invoice Level) ---")
df_h5 = df_clean.copy()
df_h5['IsReturn'] = (df_h5['Quantity'] < 0)

invoice_status = df_h5.groupby(['Country', 'InvoiceNo'])['IsReturn'].any().reset_index()
country_cross = pd.crosstab(invoice_status['Country'], invoice_status['IsReturn'])

if False in country_cross.columns:
    country_cross.rename(columns={False: 'Normal'}, inplace=True)
if True in country_cross.columns:
    country_cross.rename(columns={True: 'Return'}, inplace=True)

if 'Return' not in country_cross.columns:
    country_cross['Return'] = 0
if 'Normal' not in country_cross.columns:
    country_cross['Normal'] = 0

country_cross['Total'] = country_cross['Normal'] + country_cross['Return']
country_valid = country_cross[country_cross['Total'] >= 50].copy()

chi2, p_val, dof, expected = stats.chi2_contingency(country_valid[['Normal', 'Return']])
print(f"Countries analyzed: {len(country_valid)}")
print(f"Chi2: {chi2:.4f}, P-value: {p_val:.4e}")
print(f"Result: {'REJECT' if p_val < 0.05 else 'FAIL TO REJECT'} null hypothesis")

# 4.6 Return Rate Heatmap
return_rates = (country_valid['Return'] / country_valid['Total']).reset_index()
return_rates.columns = ['Country', 'ReturnRate']

plt.figure(figsize=(10, 6))
sns.heatmap(return_rates.pivot_table(values='ReturnRate', index='Country'),
            annot=True, cmap='Reds', fmt='.2f')
plt.title('Return Rate by Country')
plt.tight_layout()
plt.savefig('figures/return_rate_heatmap.png', dpi=300)
plt.show()

print("✓ Return rate heatmap created")



In [ ]:
'''5. Customer Segmentation via Clustering

We apply clustering on log-transformed RFM features and compare:
- K-Means
- Hierarchical clustering
- Gaussian Mixture Models

We also select an optimal **K** using the silhouette score.'''


In [ ]:
print("\n" + "="*80)
print("SECTION 5: CUSTOMER SEGMENTATION (CLUSTERING)")
print("="*80)

rfm_features = rfm[['Recency', 'Frequency', 'Monetary']]
rfm_log = np.log1p(rfm_features)
scaler = StandardScaler()
rfm_scaled = scaler.fit_transform(rfm_log)

print("Determining Optimal K...")
silhouette_scores = []
K_range = range(2, 6)

for k in K_range:
    km = KMeans(n_clusters=k, random_state=42, n_init=10)
    labels = km.fit_predict(rfm_scaled)
    score = silhouette_score(rfm_scaled, labels)
    silhouette_scores.append(score)
    print(f"K={k}, Silhouette={score:.4f}")

optimal_k = K_range[np.argmax(silhouette_scores)]
print(f"\nOptimal K based on Silhouette: {optimal_k}")

comparison_k = 2
print(f"\n--- Clustering Model Comparison (K={comparison_k}) ---")

models = {
    'K-Means': KMeans(n_clusters=comparison_k, random_state=42, n_init=10),
    'Hierarchical': AgglomerativeClustering(n_clusters=comparison_k),
    'Gaussian Mixture': GaussianMixture(n_components=comparison_k, random_state=42)
}

for name, model in models.items():
    labels = model.fit_predict(rfm_scaled)
    sil = silhouette_score(rfm_scaled, labels)
    db = davies_bouldin_score(rfm_scaled, labels)
    print(f"{name}: Silhouette={sil:.4f}, Davies-Bouldin={db:.4f}")

kmeans_final = KMeans(n_clusters=comparison_k, random_state=42, n_init=10)
rfm['Cluster'] = kmeans_final.fit_predict(rfm_scaled)

# Visualization
plt.figure(figsize=(12, 6))

# 2D view: Recency vs Monetary
plt.subplot(1, 2, 1)
scatter = plt.scatter(
    rfm_log['Recency'],
    rfm_log['Monetary'],
    c=rfm['Cluster'],
    cmap='viridis',
    alpha=0.6
)
plt.xlabel('Log Recency')
plt.ylabel('Log Monetary')
plt.title(f'Customer Clusters (K={comparison_k})')
plt.colorbar(scatter, label='Cluster')

# Normalized cluster centers
plt.subplot(1, 2, 2)
cluster_avg = rfm.groupby('Cluster')[['Recency', 'Frequency', 'Monetary']].mean()
cluster_avg_norm = (cluster_avg - cluster_avg.min()) / (cluster_avg.max() - cluster_avg.min())
cluster_avg_norm.plot(kind='bar', ax=plt.gca())
plt.title('Normalized Cluster Centers')
plt.xlabel('Cluster')
plt.xticks(rotation=0)
plt.legend(loc='upper right')

plt.tight_layout()
plt.savefig('figures/clustering_results.png', dpi=300)
plt.show()

print("\n✓ Clustering visualization created (saved to figures/)")

# 5.4 3D Cluster Visualization
fig = px.scatter_3d(
    rfm_log,
    x='Recency',
    y='Frequency',
    z='Monetary',
    color=rfm['Cluster'],
    title='3D Customer Clusters (RFM Space)'
)
fig.write_html("figures/clusters_3d.html")
fig.show()

print("✓ 3D cluster visualization created")



In [ ]:
'''6. Predictive Modeling – Churn / Retention Prediction

We use a **time-based split**:
- Train on all behavior before a cutoff (~90 days before the last date)
- Predict whether a customer will **return in the next 3 months**

Models compared:
- Logistic Regression  
- Decision Tree  
- Random Forest  
'''

In [ ]:
print("\n" + "="*80)
print("SECTION 6: PREDICTIVE MODELING - CHURN PREDICTION (CORRECTED)")
print("Methodology: Time-Series Split (Holdout Method)")
print("="*80)

# 1. Define Cutoff
max_date = df_sales['InvoiceDate'].max()
cutoff_date = max_date - pd.Timedelta(days=90)
print(f"Cutoff Date: {cutoff_date}")

# 2. Split Data
train_data = df_sales[df_sales['InvoiceDate'] < cutoff_date].copy()
test_data = df_sales[df_sales['InvoiceDate'] >= cutoff_date].copy()

# 3. Feature Engineering
X = train_data.groupby('CustomerID').agg({
    'TotalAmount': 'sum',
    'InvoiceNo': 'nunique',
    'Quantity': 'sum',
    'InvoiceDate': 'max'
}).reset_index()

X.columns = ['CustomerID', 'TotalSpent', 'Frequency', 'TotalQuantity', 'LastPurchaseDate']

first_purchase = train_data.groupby('CustomerID')['InvoiceDate'].min().reset_index()
first_purchase.columns = ['CustomerID', 'FirstPurchaseDate']
X = X.merge(first_purchase, on='CustomerID')

X['DaysSinceFirst'] = (cutoff_date - X['FirstPurchaseDate']).dt.days
X['Recency'] = (cutoff_date - X['LastPurchaseDate']).dt.days
X['AvgOrderValue'] = X['TotalSpent'] / X['Frequency']

# 4. Target Variable
future_customers = test_data['CustomerID'].unique()
X['WillReturn'] = X['CustomerID'].isin(future_customers).astype(int)

# 5. Prepare Data
feature_cols = ['TotalSpent', 'AvgOrderValue', 'TotalQuantity', 'DaysSinceFirst', 'Recency', 'Frequency']
X_model = X[feature_cols].fillna(0)
y_model = X['WillReturn']

scaler_pred = StandardScaler()
X_scaled = scaler_pred.fit_transform(X_model)

X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y_model, test_size=0.25, random_state=42, stratify=y_model
)

# 6. Models
models = {
    'Logistic Regression': LogisticRegression(random_state=42),
    'Decision Tree': DecisionTreeClassifier(max_depth=5, random_state=42),
    'Random Forest': RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42)
}

print("\n--- Model Performance (with Cross-Validation) ---")
for name, model in models.items():
    cv_scores = cross_val_score(model, X_train, y_train, cv=5, scoring='roc_auc')
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    y_prob = model.predict_proba(X_test)[:, 1]
    
    acc = accuracy_score(y_test, y_pred)
    auc = roc_auc_score(y_test, y_prob)
    
    print(f"\n{name}:")
    print(f"  CV ROC-AUC: {cv_scores.mean():.4f} (+/- {cv_scores.std():.4f})")
    print(f"  Test Accuracy: {acc:.4f}")
    print(f"  Test ROC-AUC: {auc:.4f}")

# 7. Classification Report & Feature Importance (Random Forest)
best_model = models['Random Forest']
y_pred_rf = best_model.predict(X_test)

print("\n--- Classification Report (Random Forest) ---")
print(classification_report(y_test, y_pred_rf, target_names=['Churned', 'Retained']))

importances = pd.DataFrame({
    'Feature': feature_cols,
    'Importance': best_model.feature_importances_
}).sort_values('Importance', ascending=False)

print("\n--- Feature Importance ---")
print(importances)

print("\n" + "="*80)
print("ANALYSIS COMPLETE - OUTPUTS GENERATED")
print("="*80)